# **Import all libraries**

In [2]:
import json
import pandas as pd
import numpy as np

# **Cleaning of data from coffee-desk-scraper**


In [206]:
with open('data/data_pl_coffee_all.json', 'r', encoding='ascii') as json_file:
    data = json_file.read().replace('\n', '')
    data = data.replace('[', '')
    data = data.replace(']', '')
    data = data.replace('},', '}')

    list_of_strings = data.split("{")
    list_of_strings = list_of_strings[1:] 

    list_of_corrected_strings = []
    for dict in list_of_strings:
        list_of_corrected_strings.append('{' + dict)

    list_of_dictionaries = []
    for string in list_of_corrected_strings:
        dict = json.loads(string)
        unique_id = dict['unique_id']
        try:
            price = str(dict['price'])
        except:
            price = 0
        try:
            weight = dict['Opakowanie:']
        except:
            weight = ""
        try:
            process = dict['Obróbka:']
        except:
            process = ""
        try:
            grind_types = dict['Przeznaczenie:']
        except:
            grind_types = ''
        try:
            roast = dict['Stopień palenia ziaren:']
        except:
            roast = ''
        try:
            blend = dict['Rodzaj kawy:']
        except:
            blend = ''
        try:
            description = dict['description']
        except:
            description = None
        try:
            arabica_robusta = dict['Arabica / Robusta:']
        except:
            arabica_robusta = None
        try:
            origin = dict['Pochodzenie:']
        except:
            origin = None

        coffee_dictionary = {
            "unique_id" : unique_id,
            "price" : price,
            "weight" : weight,
            "process" : process,
            "grind_types" : grind_types,
            "roast" : roast,
            "blend" : blend,
            "description" : description,
            "arabica_robusta" : arabica_robusta,
            "origin" : origin
        }
        list_of_dictionaries.append(coffee_dictionary)

In [207]:
# dictionary_test = list_of_dictionaries[0].keys()
df_coffee = pd.DataFrame(columns=['unique_id', 'price', 'weight', 'process', 'grind_types', 'roast', 'blend', 'description', 'arabica_robusta', 'origin'])

In [208]:
for idx, dict in enumerate(list_of_dictionaries): #merging dictionaries into df
        df = pd.DataFrame(dict, index=[idx])
        df_coffee = df_coffee.append(df)

In [216]:
df_coffee_without_duplicates = df_coffee.drop_duplicates(keep="first", inplace=False) # removing duplicates
df_coffee_without_duplicates.head(10)

,unique_id,price,weight,process,grind_types,roast,blend,description,arabica_robusta,origin
0,https://www.coffeedesk.pl/product/17602/Espres...,"139,00 zł",1 kg,Natural,Espresso,Średni,Ziarnista,Maj rozpoczynamy z nowym espresso miesiąca! Ty...,100% Arabica,Uganda
1,https://www.coffeedesk.pl/product/17601/Espres...,"43,00 zł",250 g,Natural,Espresso,Średni,Ziarnista,Maj rozpoczynamy z nowym espresso miesiąca! Ty...,100% Arabica,Uganda
2,https://www.coffeedesk.pl/product/115/Johan-An...,"79,00 zł",500 g,,Espresso,Średni,Ziarnista,Flagowa mieszanka ziaren jakości speciality ze...,100% Arabica,None
3,https://www.coffeedesk.pl/product/753/Arcaffe-...,"99,00 zł",1 kg,,Espresso,Średni,Ziarnista,Gorgona to nazwa jednej z wysp archipelagu Tus...,85/15,Włochy
4,https://www.coffeedesk.pl/product/754/Arcaffe-...,"32,00 zł",250 g,,Espresso,Średni,Ziarnista,Gorgona to nazwa jednej z wysp archipelagu Tus...,85/15,Włochy
5,https://www.coffeedesk.pl/product/5874/Arcaffe...,"65,00 zł",500 g,,Espresso,Średni,Ziarnista,,100% Arabica,None
6,https://www.coffeedesk.pl/product/272/Arcaffe-...,"79,00 zł",1 kg,,Espresso,Średni,Ziarnista,Arcaffe Margo to wyważona mieszanka ziaren kaw...,50/50,None
7,https://www.coffeedesk.pl/product/267/Arcaffe-...,"99,00 zł",1 kg,,Espresso,Średni,Ziarnista,Meloria to mieszanka ziaren arabiki (75%) i ro...,75/25,Włochy
8,https://www.coffeedesk.pl/product/268/Arcaffe-...,"31,90 zł",250 g,,Espresso,Średni,Ziarnista,Meloria to mieszanka ziaren arabiki (75%) i ro...,75/25,Włochy
9,https://www.coffeedesk.pl/product/269/Arcaffe-...,"125,00 zł",1 kg,,Espresso,Średni,Ziarnista,Mokacrema to blend kaw ziarnistych jakości spe...,100% Arabica,None


In [217]:
df_coffee_without_duplicates.dtypes

unique_id          object
price              object
weight             object
process            object
grind_types        object
roast              object
blend              object
description        object
arabica_robusta    object
origin             object
dtype: object

In [218]:
df_coffee_without_duplicates = df_coffee_without_duplicates.astype("str")

In [219]:
df_coffee_without_duplicates["price"] = df_coffee_without_duplicates["price"].str.strip(" zł")
df_coffee_without_duplicates["price"] = df_coffee_without_duplicates["price"].str.replace(",", ".")
df_coffee_without_duplicates["price"] = df_coffee_without_duplicates["price"].astype("float")
df_coffee_without_duplicates["price"].head()


0    139.0
1     43.0
2     79.0
3     99.0
4     32.0
Name: price, dtype: float64

In [227]:
idx = df_coffee_without_duplicates[df_coffee_without_duplicates['weight'] == '200 ml'].index
idx
for i in idx:
    df_coffee_without_duplicates = df_coffee_without_duplicates.drop(i)

In [228]:
df_coffee_without_duplicates["weight(g)"] = df_coffee_without_duplicates["weight"].str.replace("6 sztuk", "6000")
df_coffee_without_duplicates["weight(g)"] = df_coffee_without_duplicates["weight(g)"].str.replace("5 x 250", "1250")
df_coffee_without_duplicates["weight(g)"] = df_coffee_without_duplicates["weight(g)"].str.replace("8 x 11g", "88")
df_coffee_without_duplicates["weight(g)"] = df_coffee_without_duplicates["weight(g)"].str.replace("7 x 10", "70")
df_coffee_without_duplicates["weight(g)"] = df_coffee_without_duplicates["weight(g)"].str.replace("6 x 60", "360")
df_coffee_without_duplicates["weight(g)"] = df_coffee_without_duplicates["weight(g)"].str.replace("6 x 55", "330")
df_coffee_without_duplicates["weight(g)"] = df_coffee_without_duplicates["weight(g)"].str.replace("2 x 200", "400")
df_coffee_without_duplicates["weight(g)"] = df_coffee_without_duplicates["weight(g)"].str.replace("5 x 100", "400")
df_coffee_without_duplicates["weight(g)"] = df_coffee_without_duplicates["weight(g)"].str.replace("6x 1", "6000")
df_coffee_without_duplicates["weight(g)"] = df_coffee_without_duplicates["weight(g)"].str.replace("1 kg", "1000")
df_coffee_without_duplicates = df_coffee_without_duplicates[df_coffee_without_duplicates.weight != '10 saszetek']
df_coffee_without_duplicates = df_coffee_without_duplicates[df_coffee_without_duplicates.weight != '10 sztuk']

df_coffee_without_duplicates["weight(g)"].value_counts()

250 g      541
1000       163
500 g       54
200 g       50
            16
88          11
125 g       10
300 g        7
11 g         5
70g          4
252 g        4
10g          3
400g         2
6000 kg      2
6000         1
100 g        1
360g         1
330g         1
1250g        1
150 g        1
400 g        1
90 g         1
Name: weight(g), dtype: int64

In [229]:
df_coffee_without_duplicates["weight(g)"] = df_coffee_without_duplicates["weight(g)"].str.strip(" kg")
df_coffee_without_duplicates["weight(g)"] = df_coffee_without_duplicates["weight(g)"].str.strip(" g")
df_coffee_without_duplicates["weight(g)"] = df_coffee_without_duplicates["weight(g)"].fillna(0)

df_coffee_without_duplicates["weight(g)"].value_counts()

250     541
1000    163
500      54
200      50
         16
88       11
125      10
300       7
11        5
252       4
70        4
400       3
6000      3
10        3
150       1
100       1
90        1
360       1
1250      1
330       1
Name: weight(g), dtype: int64

In [230]:
df_coffee_without_duplicates["weight(g)"]

0       1000
1        250
2        500
3       1000
4        250
        ... 
4243    1000
4244     250
4245     250
4246        
4247     250
Name: weight(g), Length: 880, dtype: object

In [231]:
len(df_coffee_without_duplicates)

880

In [232]:
df_coffee_without_duplicates['roast'].value_counts()

Jasny     383
Średni    277
Ciemny    198
           22
Name: roast, dtype: int64

In [233]:
df_coffee_without_duplicates['roast'] = df_coffee_without_duplicates['roast'].str.replace("Jasny", "light")
df_coffee_without_duplicates['roast'] = df_coffee_without_duplicates['roast'].str.replace("Średni", "medium")
df_coffee_without_duplicates['roast'] = df_coffee_without_duplicates['roast'].str.replace("Ciemny", "dark")
df_coffee_without_duplicates['roast'].value_counts()

light     383
medium    277
dark      198
           22
Name: roast, dtype: int64

In [234]:
import numpy as np

In [235]:
# np.where(df_coffee_without_duplicates.applymap(lambda x: x ==''))
df_coffee_without_duplicates[df_coffee_without_duplicates["weight(g)"] == ''].index

Int64Index([ 337,  339,  342,  343,  344,  345,  346, 1737, 1738, 1739, 1756,
            2450, 2451, 2452, 3296, 4246],
           dtype='int64')

In [236]:
df_coffee_without_duplicates.loc[337,"weight(g)"] = 0 # replacing empty cells as fillna did not worl
df_coffee_without_duplicates.loc[339,"weight(g)"] = 0
df_coffee_without_duplicates.loc[342:346,"weight(g)"] = 0
df_coffee_without_duplicates.loc[1737:1739,"weight(g)"] = 0
df_coffee_without_duplicates.loc[1756,"weight(g)"] = 0
df_coffee_without_duplicates.loc[2450:2452,"weight(g)"] = 0
df_coffee_without_duplicates.loc[3296,"weight(g)"] = 0
df_coffee_without_duplicates.loc[4246,"weight(g)"] = 0

In [237]:
df_coffee_without_duplicates[df_coffee_without_duplicates["weight(g)"] == ''].index
df_coffee_without_duplicates["weight(g)"] = df_coffee_without_duplicates["weight(g)"].astype("float")

In [266]:
df_coffee_without_duplicates['price_per_kg'] = df_coffee_without_duplicates["price"].div(df_coffee_without_duplicates["weight(g)"]).mul(1000)

df_coffee_without_duplicates.head()

,unique_id,price,weight,process,grind_types,roast,blend,description,arabica_robusta,origin,weight(g),price_per_kg
0,https://www.coffeedesk.pl/product/17602/Espres...,139.0,1 kg,Natural,espresso,medium,beans,Maj rozpoczynamy z nowym espresso miesiąca! Ty...,100% Arabica,Uganda,1000.0,139.0
1,https://www.coffeedesk.pl/product/17601/Espres...,43.0,250 g,Natural,espresso,medium,beans,Maj rozpoczynamy z nowym espresso miesiąca! Ty...,100% Arabica,Uganda,250.0,172.0
2,https://www.coffeedesk.pl/product/115/Johan-An...,79.0,500 g,NaN,espresso,medium,beans,Flagowa mieszanka ziaren jakości speciality ze...,100% Arabica,None,500.0,158.0
3,https://www.coffeedesk.pl/product/753/Arcaffe-...,99.0,1 kg,NaN,espresso,medium,beans,Gorgona to nazwa jednej z wysp archipelagu Tus...,85/15,Włochy,1000.0,99.0
4,https://www.coffeedesk.pl/product/754/Arcaffe-...,32.0,250 g,NaN,espresso,medium,beans,Gorgona to nazwa jednej z wysp archipelagu Tus...,85/15,Włochy,250.0,128.0


In [268]:
df_coffee_without_duplicates["weight(g)"].describe()

count     880.000000
mean      411.842045
std       445.582347
min         0.000000
25%       250.000000
50%       250.000000
75%       500.000000
max      6000.000000
Name: weight(g), dtype: float64

In [269]:
df_coffee_without_duplicates["price"].describe()

count    880.000000
mean      66.651705
std       68.806310
min        4.500000
25%       36.000000
50%       54.000000
75%       69.925000
max      675.000000
Name: price, dtype: float64

In [267]:
df_coffee_without_duplicates['price_per_kg'].describe()

count    880.0
mean       inf
std        NaN
min       39.0
25%      115.6
50%      168.0
75%      232.0
max        inf
Name: price_per_kg, dtype: float64

In [264]:
#remove infinite values

# m = df_coffee_without_duplicates.loc[df_coffee_without_duplicates['price_per_kg'] != np.inf, 'price_per_kg'].max
# df_coffee_without_duplicates['price_per_kg'].replace(np.inf,m,inplace=True)

In [239]:
blend = set(df_coffee_without_duplicates['blend'])
blend

{'', 'Mielona', 'W kapsułkach', 'Ziarnista'}

In [240]:
df_coffee_without_duplicates['blend'] = df_coffee_without_duplicates['blend'].replace(r'^\s*$', np.nan, regex=True)

In [241]:
grind = set(df_coffee_without_duplicates['grind_types'])
grind

{'', 'Drip (Metody alternatywne)', 'Drip i espresso', 'Espresso'}

In [242]:
df_coffee_without_duplicates['grind_types'] = df_coffee_without_duplicates['grind_types'].replace(r'^\s*$', np.nan, regex=True)

In [243]:
process = set(df_coffee_without_duplicates['process'])
process

{'',
 'Aerobic Fermentation in Piles',
 'Anaerobic',
 'Anaerobic natural',
 'Black Honey',
 'CRYO',
 'Carbonic Maceration',
 'Closed Tank Fermentation',
 'Experimental',
 'Experimental Anaerobic',
 'Fermentacja kontrolowana',
 'Honey',
 'Honey + Anaerobic',
 'Hybrid Natural',
 'Macerated Natural',
 'Monsooning',
 'Natural',
 'Pulped natural',
 'Red Honey',
 'Rum Aged',
 'Semi-Carbonic Maceration',
 'Semi-Washed Kombucha Experiment',
 'Semi-washed',
 'Washed',
 'Washed Anaerobic',
 'Washed Double Anaerobic',
 'Washed i natural',
 'Washed, Barrel Aged',
 'Wet-hulling',
 'Yeast Process',
 'Yellow Bourbon',
 'Yellow Honey'}

In [244]:
df_coffee_without_duplicates['process'] = df_coffee_without_duplicates['process'].replace(r'^\s*$', np.nan, regex=True)
process = set(df_coffee_without_duplicates['process'])
process

{'Aerobic Fermentation in Piles',
 'Anaerobic',
 'Anaerobic natural',
 'Black Honey',
 'CRYO',
 'Carbonic Maceration',
 'Closed Tank Fermentation',
 'Experimental',
 'Experimental Anaerobic',
 'Fermentacja kontrolowana',
 'Honey',
 'Honey + Anaerobic',
 'Hybrid Natural',
 'Macerated Natural',
 'Monsooning',
 'Natural',
 'Pulped natural',
 'Red Honey',
 'Rum Aged',
 'Semi-Carbonic Maceration',
 'Semi-Washed Kombucha Experiment',
 'Semi-washed',
 'Washed',
 'Washed Anaerobic',
 'Washed Double Anaerobic',
 'Washed i natural',
 'Washed, Barrel Aged',
 'Wet-hulling',
 'Yeast Process',
 'Yellow Bourbon',
 'Yellow Honey',
 nan}

In [245]:
df_coffee_without_duplicates.head()

,unique_id,price,weight,process,grind_types,roast,blend,description,arabica_robusta,origin,weight(g),price_per_kg
0,https://www.coffeedesk.pl/product/17602/Espres...,139.0,1 kg,Natural,Espresso,medium,Ziarnista,Maj rozpoczynamy z nowym espresso miesiąca! Ty...,100% Arabica,Uganda,1000.0,139.0
1,https://www.coffeedesk.pl/product/17601/Espres...,43.0,250 g,Natural,Espresso,medium,Ziarnista,Maj rozpoczynamy z nowym espresso miesiąca! Ty...,100% Arabica,Uganda,250.0,172.0
2,https://www.coffeedesk.pl/product/115/Johan-An...,79.0,500 g,NaN,Espresso,medium,Ziarnista,Flagowa mieszanka ziaren jakości speciality ze...,100% Arabica,None,500.0,158.0
3,https://www.coffeedesk.pl/product/753/Arcaffe-...,99.0,1 kg,NaN,Espresso,medium,Ziarnista,Gorgona to nazwa jednej z wysp archipelagu Tus...,85/15,Włochy,1000.0,99.0
4,https://www.coffeedesk.pl/product/754/Arcaffe-...,32.0,250 g,NaN,Espresso,medium,Ziarnista,Gorgona to nazwa jednej z wysp archipelagu Tus...,85/15,Włochy,250.0,128.0


In [252]:
idx = df_coffee_without_duplicates[df_coffee_without_duplicates['blend'] == "W kapsułkach"].index
df_coffee_without_duplicates = df_coffee_without_duplicates.drop(idx)


In [254]:
df_coffee_without_duplicates['blend'] = df_coffee_without_duplicates['blend'].str.replace("Mielona", "ground")
df_coffee_without_duplicates['blend'] = df_coffee_without_duplicates['blend'].str.replace("Ziarnista", "beans")
df_coffee_without_duplicates['blend'].value_counts()

beans       773
ground       82
capsules      1
Name: blend, dtype: int64

In [260]:
df_coffee_without_duplicates['description'] = df_coffee_without_duplicates['description'].replace(r'^\s*$', np.nan, regex=True)

In [255]:
df_coffee_without_duplicates['grind_types'] = df_coffee_without_duplicates['grind_types'].str.replace("Drip (Metody alternatywne)", "drip")
df_coffee_without_duplicates['grind_types'] = df_coffee_without_duplicates['grind_types'].str.replace("Drip i espresso", "drip, espresso")
df_coffee_without_duplicates['grind_types'] = df_coffee_without_duplicates['grind_types'].str.replace("Espresso", "espresso")
df_coffee_without_duplicates['grind_types'].value_counts()

espresso                      401
Drip (Metody alternatywne)    382
drip, espresso                 72
Name: grind_types, dtype: int64

In [257]:
df_coffee_without_duplicates.to_csv('data/pl_coffees.csv')

In [261]:
df_coffee_without_duplicates[['price','process', 'grind_types',	'roast', 'blend', 'description', 'arabica_robusta',	'origin', 'weight(g)', 'price_per_kg']].head(20)

,price,process,grind_types,roast,blend,description,arabica_robusta,origin,weight(g),price_per_kg
0,139.0,Natural,espresso,medium,beans,Maj rozpoczynamy z nowym espresso miesiąca! Ty...,100% Arabica,Uganda,1000.0,139.0
1,43.0,Natural,espresso,medium,beans,Maj rozpoczynamy z nowym espresso miesiąca! Ty...,100% Arabica,Uganda,250.0,172.0
2,79.0,NaN,espresso,medium,beans,Flagowa mieszanka ziaren jakości speciality ze...,100% Arabica,None,500.0,158.0
3,99.0,NaN,espresso,medium,beans,Gorgona to nazwa jednej z wysp archipelagu Tus...,85/15,Włochy,1000.0,99.0
4,32.0,NaN,espresso,medium,beans,Gorgona to nazwa jednej z wysp archipelagu Tus...,85/15,Włochy,250.0,128.0
5,65.0,NaN,espresso,medium,beans,NaN,100% Arabica,None,500.0,130.0
6,79.0,NaN,espresso,medium,beans,Arcaffe Margo to wyważona mieszanka ziaren kaw...,50/50,None,1000.0,79.0
7,99.0,NaN,espresso,medium,beans,Meloria to mieszanka ziaren arabiki (75%) i ro...,75/25,Włochy,1000.0,99.0
8,31.9,NaN,espresso,medium,beans,Meloria to mieszanka ziaren arabiki (75%) i ro...,75/25,Włochy,250.0,127.6
9,125.0,NaN,espresso,medium,beans,Mokacrema to blend kaw ziarnistych jakości spe...,100% Arabica,None,1000.0,125.0


# **Clening of data from brewed-scraper**

In [278]:
with open('data/data_brew.json', 'r', encoding='ascii') as json_file2:
    data = json_file2.read().replace('\n', '')
    data = data.replace('[', '')
    data = data.replace(']', '')
    data = data.replace('},', '}')

    list_of_strings = data.split("{")
    list_of_strings = list_of_strings[1:]
    # print(list_of_strings[0])

    list_of_dictionaries = []
    for string in list_of_strings:
        dict = json.loads(string)
        try:
            price = str(dict['price'])
        except:
            price = "0"
        try:
            weight = dict['weight']
        except:
            weight = None
        try:
            process = dict['process']
        except:
            process = None
        try:
            grind_types = dict['grind_types']
        except:
            grind_types = None
        roast = 'No info'
        blend = 'No info'
        try:
            description = dict['flavor']
        except:
            description = None
        arabica_robusta = 'No info'
        try:
            origin = dict['origin']
        except:
            origin = None

        coffee_dictionary = {
            "unique_id" : unique_id,
            "price" : price,
            "weight" : weight,
            "process" : process,
            "grind_types" : grind_types,
            "roast" : roast,
            "blend" : blend,
            "description" : description,
            "arabica_robusta" : arabica_robusta,
            "origin" : origin
        }
        list_of_dictionaries.append(coffee_dictionary)

JSONDecodeError: Extra data: line 1 column 18 (char 17)